In [1]:
import numpy as np
from qutip import (
    basis, tensor, Qobj, ket2dm,
    sigmax, sigmay, sigmaz, expect,identity
)

In [2]:
def depolarizing_channel(rho, p):
    """
    Apply single-qubit depolarizing noise to both qubits in a 2-qubit system.
    Returns the noisy 2-qubit density matrix.
    """
    # Pauli matrices
    I = identity(2)
    X = sigmax()
    Y = sigmay()
    Z = sigmaz()

    # Apply noise to both qubits independently
    rho_noisy = (1 - p) * rho

    # Apply each Pauli to each qubit (separately)
    for P in [X, Y, Z]:
        rho_noisy += (p / 3) * (
            tensor(P, I) * rho * tensor(P, I).dag() +
            tensor(I, P) * rho * tensor(I, P).dag()
        )

    return rho_noisy


In [ ]:

def normalize_state(state):
    """Ensures the state is normalized."""
    return state / state.norm()

def singlet_state():

    zero = basis(2, 0)
    one  = basis(2, 1)
    # Construct the singlet
    psi_m = (tensor(zero, one) - tensor(one, zero)).unit()
    return psi_m


def spin_operator(theta, phi):
    nx = np.sin(theta) * np.cos(phi)
    ny = np.sin(theta) * np.sin(phi)
    nz = np.cos(theta)
    return nx * sigmax() + ny * sigmay() + nz * sigmaz()

def projectors_from_operator(op):
    evals, evecs = op.eigenstates()
    # Typically, for a spin-1/2 measurement, eigenvalues are +1, -1
    projector_p = ket2dm(evecs[0])  # +1 eigenvector
    projector_m = ket2dm(evecs[1])  # -1 eigenvector
    return projector_p, projector_m

def generate_measurement_bases():
    # Use shared labels for both Alice and Bob
    common_labels = ["Z", "X", "Y"]

    A_ops = [sigmaz(), sigmax(), sigmay()]
    B_ops = [sigmaz(), sigmax(), sigmay()]

    Alice_bases = {}
    Bob_bases = {}

    for label, A_op, B_op in zip(common_labels, A_ops, B_ops):
        Alice_bases[label] = projectors_from_operator(A_op)
        Bob_bases[label] = projectors_from_operator(B_op)

    return Alice_bases, Bob_bases

# -- 3. Run a simulation for N entangled pairs --

def simulate_e91_protocol(N):
    # Generate measurement bases
    Alice_bases, Bob_bases = generate_measurement_bases()
    A_labels = list(Alice_bases.keys())
    B_labels = list(Bob_bases.keys())

    outcomes = []
    # For each run, prepare a new singlet state
    for _ in range(N):
        psi = singlet_state()
        rho = ket2dm(psi)  # convert pure state to density matrix

        # Apply depolarizing noise with probability p
        p_noise = 0.5 # 5% noise
        rho_noisy = depolarizing_channel(rho, p_noise)

        # Randomly choose one of the three bases for Alice and Bob
        A_choice = np.random.choice(A_labels)
        B_choice = np.random.choice(B_labels)

        # Each basis has P+ and P- projectors
        A_proj_plus, A_proj_minus = Alice_bases[A_choice]
        B_proj_plus, B_proj_minus = Bob_bases[B_choice]

        
        P_Aplus_Bplus = tensor(A_proj_plus, B_proj_plus)
        P_Aplus_Bminus = tensor(A_proj_plus, B_proj_minus)
        P_Aminus_Bplus = tensor(A_proj_minus, B_proj_plus)
        P_Aminus_Bminus = tensor(A_proj_minus, B_proj_minus)

        p_aplus_bplus  = (P_Aplus_Bplus  * rho_noisy).tr().real
        p_aplus_bminus = (P_Aplus_Bminus * rho_noisy).tr().real
        p_aminus_bplus = (P_Aminus_Bplus * rho_noisy).tr().real
        p_aminus_bminus= (P_Aminus_Bminus* rho_noisy).tr().real



        # Pick an outcome according to these probabilities
        # (Pseudo “collapse” step in a simulation)
        outcome_probs = [p_aplus_bplus, p_aplus_bminus, p_aminus_bplus, p_aminus_bminus]
        total_prob = sum(outcome_probs)
        outcome_probs = [p / total_prob for p in outcome_probs]
        outcome_index = np.random.choice([0,1,2,3], p=outcome_probs)

        if outcome_index == 0:
            A_result, B_result = +1, +1
        elif outcome_index == 1:
            A_result, B_result = +1, -1
        elif outcome_index == 2:
            A_result, B_result = -1, +1
        else:
            A_result, B_result = -1, -1

        outcomes.append((A_choice, B_choice, A_result, B_result))

    return outcomes

# Run the simulation
if __name__ == "__main__":
    N =127  # number of entangled pairs
    results = simulate_e91_protocol(N)
    same_basis_outcomes = [(A_choice, A_res, B_res)
                           for (A_choice, B_choice, A_res, B_res) in results
                           if A_choice == B_choice]

    raw_key_A = []
    raw_key_B = []
    for (basis, A_res, B_res) in same_basis_outcomes:
        bit_A = 0 if A_res == +1 else 1
        bit_B = 0 if B_res == +1 else 1
        bit_B = 1 - bit_B  # ⬅️ Apply NOT gate to Bob’s bit
        raw_key_A.append(bit_A)
        raw_key_B.append(bit_B)
    print(raw_key_A)
    print(raw_key_B)
    # Estimate how often they match
    matches = sum(a == b for a, b in zip(raw_key_A, raw_key_B))
    # errors = len(raw_key_A) - matches
    errors = sum(a != b for a, b in zip(raw_key_A, raw_key_B))
    if len(raw_key_A) > 0:
        qber = errors / len(raw_key_A) if raw_key_A else 0.0
    else:
        qber = 0.0

    if len(raw_key_A) > 0:
        match_fraction = matches / len(raw_key_A)
    else:
        match_fraction = 0.0

    print(f"Number of trials: {N}")
    print(f"Number of same-basis rounds: {len(raw_key_A)}")
    print(f"Match fraction in same-basis rounds: {match_fraction:.2f}")
    print("Alice's raw key (first 20 bits):", raw_key_A[:20])
    print("Bob's   raw key (first 20 bits):", raw_key_B[:20])
    print(f"Quantum Bit Error Rate (QBER): {qber:.2%}")


[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0]
Number of trials: 127
Number of same-basis rounds: 51
Match fraction in same-basis rounds: 0.51
Alice's raw key (first 20 bits): [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1]
Bob's   raw key (first 20 bits): [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1]
Quantum Bit Error Rate (QBER): 49.02%
